# 📚 HIST 213 Platebook Generator

**Step 1:** Run the cell below to install the PDF library (`reportlab`).
**Step 2:** Run the second cell to generate your platebook.

In [ ]:
# Install required library
!pip install reportlab

In [ ]:
# @title Generate Platebook
# @markdown Enter your published Google Sheet CSV URL below and press the Play button.

GOOGLE_SHEET_CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRM99Thl9mOl_ua5_oOA5ymxIpPEsKrW6hCOXVlsI4NInnU8g4GpYl6Tomc2PtcqRo0LGkSHholSt5b/pub?output=csv" # @param {type:"string"}
COURSE_NAME = "HIST 213 East Asia in the Modern World" # @param {type:"string"}
TERM = "Winter 2026" # @param {type:"string"}
OUTPUT_FILENAME = "HIST213_Platebook_Winter2026.pdf" # @param {type:"string"}

import sys
import json
import requests
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.colors import Color, black
from google.colab import files

# =============================================================================
# PAGE + DESIGN CONSTANTS
# =============================================================================

PAGE_WIDTH, PAGE_HEIGHT = letter
LEFT_MARGIN = 36
RIGHT_MARGIN = 576
TOP_MARGIN = 29

LIGHT_GRAY = Color(0.933333, 0.933333, 0.933333)
WHITE = Color(1, 1, 1)
BLACK = black

CORNER_RADIUS = 5
FONT_NAME = "Helvetica"
FONT_SIZE = 11

# Layout
HEADER_Y = 29
HEADER_HEIGHT = 28
PLATE_NUM_X = 36
PLATE_NUM_WIDTH = 85
TITLE_X = 125
TITLE_WIDTH = 280
DATE_X = 465
DATE_WIDTH = 111

PPT_LABEL_Y = 68
PPT_LABEL_HEIGHT = 24
PERSON_LABEL_X = 36
PERSON_LABEL_WIDTH = 65
PLACE_LABEL_X = 216
PLACE_LABEL_WIDTH = 55
THING_LABEL_X = 396
THING_LABEL_WIDTH = 60

PPT_CONTENT_START_Y = 92
PPT_CONTENT_HEIGHT = 20
PPT_CONTENT_WIDTH = 180
PPT_ROWS = 3

TIMELINE_LABEL_Y = 158
TIMELINE_LABEL_HEIGHT = 24
TIMELINE_LABEL_WIDTH = 70
TIMELINE_BOX_Y = 182
TIMELINE_BOX_HEIGHT = 60
TIMELINE_BOX_WIDTH = 540
TIMELINE_LINE_Y = 212
TICK_START_X = 50
TICK_END_X = 562
TICK_COUNT = 11

MAP_LABEL_Y = 248
MAP_LABEL_HEIGHT = 24
MAP_LABEL_WIDTH = 45
MAP_BOX_Y = 272
MAP_BOX_HEIGHT = 200
MAP_BOX_WIDTH = 540

PQ_LABEL_Y = 483
PQ_LABEL_HEIGHT = 24
PQ_LABEL_WIDTH = 165
VSA_LABEL_X = 410
VSA_LABEL_WIDTH = 166

Q_ROW_START_Y = 507
Q_ROW_HEIGHT = 50
Q_NUM_WIDTH = 30
Q_CONTENT_WIDTH = 330
Q_ANSWER_WIDTH = 180

CEC_LABEL_Y = 622
CEC_LABEL_HEIGHT = 20
CEC_LABEL_WIDTH = 185
NOTES_LABEL_X = 312
NOTES_LABEL_WIDTH = 55

BOTTOM_BOX_Y = 642
BOTTOM_BOX_HEIGHT = 88
BOTTOM_LEFT_WIDTH = 264
BOTTOM_RIGHT_WIDTH = 264
BOTTOM_RIGHT_X = 312

# =============================================================================
# DRAWING HELPERS
# =============================================================================

def _y(top, height):
    return PAGE_HEIGHT - top - height

def draw_round(c, x, top, w, h, fill=False, gray=False):
    y = _y(top, h)
    c.setLineWidth(0.5)
    c.setStrokeColor(BLACK)
    if fill:
        if gray:
            c.setFillColor(LIGHT_GRAY)
        else:
            c.setFillColor(WHITE)
        c.roundRect(x, y, w, h, CORNER_RADIUS, stroke=1, fill=1)
    else:
        c.roundRect(x, y, w, h, CORNER_RADIUS, stroke=1, fill=0)

def draw_centered(c, text, x, top, w, h, size=FONT_SIZE, bold=False):
    font = "Helvetica-Bold" if bold else FONT_NAME
    c.setFont(font, size)
    c.setFillColor(BLACK)
    tw = c.stringWidth(text, font, size)
    y = _y(top, h) + (h - size) / 2 + 2
    c.drawString(x + (w - tw) / 2, y, text)

# =============================================================================
# PLATE DRAWING
# =============================================================================

def draw_standard_plate(c, n, title, date):
    # Header
    draw_round(c, PLATE_NUM_X, HEADER_Y, PLATE_NUM_WIDTH, HEADER_HEIGHT)
    draw_centered(c, f"Plate # {n}", PLATE_NUM_X, HEADER_Y, PLATE_NUM_WIDTH, HEADER_HEIGHT, size=12, bold=True)

    draw_round(c, TITLE_X, HEADER_Y, TITLE_WIDTH, HEADER_HEIGHT)
    draw_centered(c, title, TITLE_X, HEADER_Y, TITLE_WIDTH, HEADER_HEIGHT, size=11, bold=True)

    draw_round(c, DATE_X, HEADER_Y, DATE_WIDTH, HEADER_HEIGHT)
    draw_centered(c, date, DATE_X, HEADER_Y, DATE_WIDTH, HEADER_HEIGHT, size=12, bold=True)

    # Labels
    draw_round(c, PERSON_LABEL_X, PPT_LABEL_Y, PERSON_LABEL_WIDTH, PPT_LABEL_HEIGHT)
    draw_centered(c, "Person", PERSON_LABEL_X, PPT_LABEL_Y, PERSON_LABEL_WIDTH, PPT_LABEL_HEIGHT, size=10, bold=True)

    draw_round(c, PLACE_LABEL_X, PPT_LABEL_Y, PLACE_LABEL_WIDTH, PPT_LABEL_HEIGHT)
    draw_centered(c, "Place", PLACE_LABEL_X, PPT_LABEL_Y, PLACE_LABEL_WIDTH, PPT_LABEL_HEIGHT, size=10, bold=True)

    draw_round(c, THING_LABEL_X, PPT_LABEL_Y, THING_LABEL_WIDTH, PPT_LABEL_HEIGHT)
    draw_centered(c, "Thing", THING_LABEL_X, PPT_LABEL_Y, THING_LABEL_WIDTH, PPT_LABEL_HEIGHT, size=10, bold=True)

    # Grid
    for row in range(PPT_ROWS):
        y_pos = PPT_CONTENT_START_Y + row * PPT_CONTENT_HEIGHT
        for x in (36, 216, 396):
            draw_round(c, x, y_pos, PPT_CONTENT_WIDTH, PPT_CONTENT_HEIGHT, fill=True, gray=True)

    # Timeline
    draw_round(c, LEFT_MARGIN, TIMELINE_LABEL_Y, TIMELINE_LABEL_WIDTH, TIMELINE_LABEL_HEIGHT)
    draw_centered(c, "Timeline", LEFT_MARGIN, TIMELINE_LABEL_Y, TIMELINE_LABEL_WIDTH, TIMELINE_LABEL_HEIGHT, size=10, bold=True)
    draw_round(c, LEFT_MARGIN, TIMELINE_BOX_Y, TIMELINE_BOX_WIDTH, TIMELINE_BOX_HEIGHT, fill=True, gray=True)

    y = PAGE_HEIGHT - TIMELINE_LINE_Y
    c.line(TICK_START_X, y, TICK_END_X, y)
    step = (TICK_END_X - TICK_START_X) / TICK_COUNT
    for i in range(TICK_COUNT + 1):
        x = TICK_START_X + i * step
        c.line(x, y - 8, x, y + 8)

    # Map
    draw_round(c, LEFT_MARGIN, MAP_LABEL_Y, MAP_LABEL_WIDTH, MAP_LABEL_HEIGHT)
    draw_centered(c, "Map", LEFT_MARGIN, MAP_LABEL_Y, MAP_LABEL_WIDTH, MAP_LABEL_HEIGHT, size=10, bold=True)
    draw_round(c, LEFT_MARGIN, MAP_BOX_Y, MAP_BOX_WIDTH, MAP_BOX_HEIGHT, fill=True, gray=True)

    # Questions
    draw_round(c, LEFT_MARGIN, PQ_LABEL_Y, PQ_LABEL_WIDTH, PQ_LABEL_HEIGHT)
    draw_centered(c, "Penetrating Questions", LEFT_MARGIN, PQ_LABEL_Y, PQ_LABEL_WIDTH, PQ_LABEL_HEIGHT, size=9, bold=True)
    draw_round(c, VSA_LABEL_X, PQ_LABEL_Y, VSA_LABEL_WIDTH, PQ_LABEL_HEIGHT)
    draw_centered(c, "Very short answers", VSA_LABEL_X, PQ_LABEL_Y, VSA_LABEL_WIDTH, PQ_LABEL_HEIGHT, size=9, bold=True)

    for i in range(2):
        y = Q_ROW_START_Y + i * 60
        draw_round(c, LEFT_MARGIN, y, Q_NUM_WIDTH, Q_ROW_HEIGHT)
        draw_centered(c, str(i + 1), LEFT_MARGIN, y, Q_NUM_WIDTH, Q_ROW_HEIGHT, size=10, bold=True)
        draw_round(c, LEFT_MARGIN + Q_NUM_WIDTH, y, Q_CONTENT_WIDTH, Q_ROW_HEIGHT, fill=True, gray=True)
        draw_round(c, 396, y, Q_ANSWER_WIDTH, Q_ROW_HEIGHT, fill=True, gray=True)

    # Bottom
    draw_round(c, LEFT_MARGIN, CEC_LABEL_Y, CEC_LABEL_WIDTH, CEC_LABEL_HEIGHT)
    draw_centered(c, "Causes / Effects / Connections", LEFT_MARGIN, CEC_LABEL_Y, CEC_LABEL_WIDTH, CEC_LABEL_HEIGHT, size=8, bold=True)
    draw_round(c, NOTES_LABEL_X, CEC_LABEL_Y, NOTES_LABEL_WIDTH, CEC_LABEL_HEIGHT)
    draw_centered(c, "Notes", NOTES_LABEL_X, CEC_LABEL_Y, NOTES_LABEL_WIDTH, CEC_LABEL_HEIGHT, size=10, bold=True)
    draw_round(c, LEFT_MARGIN, BOTTOM_BOX_Y, BOTTOM_LEFT_WIDTH, BOTTOM_BOX_HEIGHT, fill=True, gray=True)
    draw_round(c, BOTTOM_RIGHT_X, BOTTOM_BOX_Y, BOTTOM_RIGHT_WIDTH, BOTTOM_BOX_HEIGHT, fill=True, gray=True)

def main():
    print("Fetching data...")
    response = requests.get(GOOGLE_SHEET_CSV_URL)
    csv_text = response.text

    lessons = []
    for line in csv_text.strip().split('\n')[1:]:
        if not line.strip(): continue
        p = line.split(',')
        if len(p) >= 3:
           lessons.append({
               "plate_number": int(p[0].strip()),
               "date": p[1].strip().replace('"', ''),
               "title": p[2].strip().replace('"', '')
           })

    print(f"Found {len(lessons)} lessons. Generating PDF...")
    c = canvas.Canvas(OUTPUT_FILENAME, pagesize=letter)

    # Cover
    c.setFont("Helvetica-Bold", 18)
    w = c.stringWidth(COURSE_NAME, "Helvetica-Bold", 18)
    c.drawString((PAGE_WIDTH - w) / 2, 180, COURSE_NAME)
    c.setFont("Helvetica-Bold", 16)
    w = c.stringWidth(TERM, "Helvetica-Bold", 16)
    c.drawString((PAGE_WIDTH - w) / 2, 150, TERM)
    c.setFont(FONT_NAME, 12)
    name = "Name:_____________________________________"
    w = c.stringWidth(name, FONT_NAME, 12)
    c.drawString((PAGE_WIDTH - w) / 2, 100, name)
    c.showPage()

    # Table of Contents
    c.setFont(FONT_NAME, 16)
    c.drawString((PAGE_WIDTH - c.stringWidth("Table of Contents", FONT_NAME, 16)) / 2, PAGE_HEIGHT - 60, "Table of Contents")
    c.setLineWidth(2)
    c.line(LEFT_MARGIN, PAGE_HEIGHT - 75, RIGHT_MARGIN, PAGE_HEIGHT - 75)

    c.setFont(FONT_NAME, 10)
    y_pos = PAGE_HEIGHT - 100
    for lesson in lessons:
        if y_pos < 100:
            c.showPage()
            y_pos = PAGE_HEIGHT - 60
        c.drawString(LEFT_MARGIN, y_pos, f"Plate {lesson['plate_number']}")
        c.drawString(LEFT_MARGIN + 80, y_pos, lesson['title'])
        c.drawString(RIGHT_MARGIN - c.stringWidth(lesson['date'], FONT_NAME, 10), y_pos, lesson['date'])
        c.setDash(1, 2); c.setLineWidth(0.5); c.line(LEFT_MARGIN, y_pos - 2, RIGHT_MARGIN, y_pos - 2); c.setDash()
        y_pos -= 20
    c.showPage()

    # Plates
    for lesson in lessons:
        draw_standard_plate(c, lesson["plate_number"], lesson["title"], lesson["date"])
        c.showPage()

    c.save()
    print("Success! Downloading PDF...")
    files.download(OUTPUT_FILENAME)

main()